In [1]:
from selenium import webdriver
import time
import csv
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import datetime
from Database import database

database.createTableScraping()
            
def convertToNumber(txt):
    numbers = re.findall(r'\d+', txt)
    return int(''.join(numbers[ : ]))

def convertToDate(posted):
    date_time_str = posted
    date_time_obj = datetime.datetime.strptime(date_time_str, '%b %d, %Y')
    date = date_time_obj.date()
    return date

playlist_dict = {'kompastv_maret' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRllutGYG7M7o4cgLNklkNcl',
                 'kompastv_april' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRkqWIyvyoa0YIoTaumlH3tg',
                 'kompastv_mei' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRl4eLvGIpYvHqUZaXfREyfq',
                 'kompastv_juni' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRlp2KPSpqjqgMxyyZEpFbKS',
                 'kompastv_juli' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRm9e-pBRy0_DoefcPImzKu6',
                 'kompastv_agustus' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRmAcdJy6ayGlRv1ApO81eA6',
                 'kompastv_september' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRklqLdrocj5lu77uj1ZrmmZ',
                 'cnnindonesia_maret' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRmddFVOIu0i8b5IEvTVxxOn',
                 'cnnindonesia_april' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRksTv4u3JDTS0UkKng43Gvq',
                 'cnnindonesia_mei' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRlU8qdT22K62kVvgFcyG18n',
                 'cnnindonesia_juni' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRluCn6pQtjqisslNroIpt8r',
                 'cnnindonesia_juli' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRkSKFZzneilqD5XYSAoajBO',
                 'cnnindonesia_agustus' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRmW-2FQMhqE9F-OSblNpfdY',
                 'cnnindonesia_september' : 'https://www.youtube.com/playlist?list=PLxJWSprwHRRl1oCCimK7awoxIKY-d6iiZ'}

playlist_name = []
playlist_links = []
for x,y in playlist_dict.items():
    playlist_name.append(x) 
    playlist_links.append(y)
    
href = [] # link video berdasarkan bulan
for x in range(len(playlist_links)):
    youtube_playlist = playlist_links[x]
    locationOfWebdriver = "d:\\chromedriver.exe"

    driver = webdriver.Chrome(locationOfWebdriver)
    driver.get(youtube_playlist)
    #driver.maximize_window()
    time.sleep(2)
    
    url = [] # link dari masing-masing video per 1 bulan

    try:
        content = driver.find_elements_by_xpath('//*[@id="content"]/a')
        tcount = 0
        #href = []
        #href.append(content[tcount].get_attribute('href'))
        while tcount < len(content):
            url.append(content[tcount].get_attribute('href'))
            tcount = tcount +1
        
    except Exception as e:
        print(e)
        driver.close()
    driver.close()
    href.append(url)
    
# STEP 3 : MEMBUKA MASING-MASING LINK DARI KUMPULAN LIST LINK PER BULAN

for j in range(len(href)): # buka urutan playlist bulan
    scraping = []
    for i in range(len(href[j])): # buka urutan playlist hari
        nama = []
        komen = []
    
        youtube_pages = href[j][i]
        locationOfWebdriver = "d:\\chromedriver.exe"

        videoData = pd.Series(dtype='object')

        driver = webdriver.Chrome(locationOfWebdriver)
        driver.get(youtube_pages)
        driver.maximize_window()
        time.sleep(3)

# STEP 4 : Extract the elements

        try: 
            vtitle = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
            subscription = driver.find_element_by_xpath('//*[@id="text"]/a').text
            num_views = driver.find_element_by_xpath('//*[@id="count"]/yt-view-count-renderer/span[1]').text
            views = convertToNumber(num_views) # Mengubah teks jumlah views menjadi angka saja
            posted = driver.find_element_by_xpath('//*[@id="date"]/yt-formatted-string').text
            tanggal = convertToDate(posted)
            comment_section = driver.find_element_by_xpath('//*[@id="comments"]') # Mencari bagian komen
        except exceptions.NoSuchElementException:
            # Note: Youtube may have changed their HTML layouts for
            # videos, so raise an error for sanity sake in case the
            # elements provided cannot be found anymore.
            error = "Error: Double check selector OR "
            error += "element may not yet be on the screen at the time of the find operation"
            print(error)
    

# STEP 5 : SCROLL HALAMAN UNTUK MEMUAT SELURUH KOMEN 
    
        # Scroll into view the comment section, then allow some time for everything to be loaded as necessary.
        driver.execute_script("arguments[0].scrollIntoView();", comment_section)
        time.sleep(5)

        # Scroll all the way down to the bottom in order to get all the elements loaded (since Youtube dynamically loads them).
        last_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            # Scroll down 'til "next load".
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

            # Wait to load everything thus far.
            time.sleep(2)

            # Calculate new scroll height and compare with last scroll height.
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # One last scroll just in case.
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)

# STEP 6 : MENDAPATKAN SELURUH USERNAME DAN KOMEN

        try:
            # Extract the elements storing the usernames and comments.
            username_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
            comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
        except exceptions.NoSuchElementException:
            error = "Error: Double check selector OR "
            error += "element may not yet be on the screen at the time of the find operation"
            print(error)
    
        for username, comment in zip(username_elems, comment_elems):
            nama_pengguna = username.text
            komen_pengguna = comment.text.replace('\n','')
            nama.append(nama_pengguna)
            komen.append(komen_pengguna)

# STEP 7 : MENG-EXPORT SELURUH METADATA KE DALAM DATAFRAME DARI TIAP VIDEO (HARI)

        comment_dict = {'nama_user':nama, 'komen_user':komen, 'judul_video':vtitle, 'channel':subscription, 'views':views, 'tanggal':tanggal}
        df = pd.DataFrame(comment_dict,columns = ['nama_user','komen_user','judul_video','channel','views','tanggal'])
        scraping.append(df)
        time.sleep(5)
        driver.close()

# STEP 8 : MENG-EXPORT KUMPULAN LIST DATAFRAME PER BULAN KE CSV        
    playlist = playlist_name[j]
    scraping = pd.concat(scraping)
    scraping.to_csv(path_data + 'scraping_youtube_%s.csv' % playlist, sep=',', index=False, encoding = "UTF-8")

# STEP 9 : LOAD TO POSTGRESQL    
    database.intoDatabaseScraping(playlist)
    print('Scraping youtube comments : %s' % playlist)

Connecting to the PostgreSQL database...
Connection successful
Table created successfully


KeyboardInterrupt: 